<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/Assessment_Mixtral_8x7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment on Mixtral 8x7B

* https://mistral.ai/news/mixtral-of-experts/
* https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1
* https://huggingface.co/mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ
* https://huggingface.co/docs/transformers/v4.42.0/quantization/hqq

### Prerequisites
1. a Huggingface account and an
1. access tokens to put in below while logging in
1. patience: this will take 10-15 minutes to load libs and model and even generation will be really slow

### Why Mixtral
* explicitly tuned for European languages (like French, Italian, German and Spanish)
* performance close to GPT 3.5
* even quantized performance is good
* only uses fraction of parameters at a time
* theoretically able to unload parameters to CPU

### Comparing GPU microarchitectures
* T4/RTX 20: https://en.wikipedia.org/wiki/Turing_(microarchitecture)
  * V100 - professional variant of RTX 20 consumer line: https://en.wikipedia.org/wiki/Volta_(microarchitecture)
* A100/RTX 30: https://en.wikipedia.org/wiki/Ampere_(microarchitecture)
* L4/L40/RTX 40: https://en.wikipedia.org/wiki/Ada_Lovelace_(microarchitecture)
  * H100 - professional variant of RTX 40 consumer line, not available on Colab (yet?): https://en.wikipedia.org/wiki/Hopper_(microarchitecture)
* Future successor to both Hopper and Ada Lovelace: https://en.wikipedia.org/wiki/Blackwell_(microarchitecture)
* comparing GPUs: https://www.reddit.com/r/learnmachinelearning/comments/18gn1b2/choosing_the_right_gpu_for_your_workloads_a_dive/


In [1]:
%%time

!pip install --upgrade -q transformers accelerate flash_attn torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 2.4.0 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but

In [2]:
%%time

# this will take a few minutes
!pip install hqq -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 87.2 MB/s eta 0:00:00
CPU times: user 805 ms, sys: 113 ms, total: 918 ms
Wall time: 2min 31s


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from IPython.display import Markdown

In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

In [6]:
!nvidia-smi

Fri Aug 23 15:32:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   49C    P8              17W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [24]:
kind = 'Mixtral-8x7B'

lang = "de"
# lang = "en"


In [7]:
%%time

import transformers
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
from hqq.core.quantize import *

model_id = 'mobiuslabsgmbh/Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = HQQModelForCausalLM.from_quantized(model_id,
                                           device_map="cuda)

HQQLinear.set_backend(HQQBackend.ATEN_BACKPROP)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

...

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.62k [00:00<?, ?B/s]

qmodel.pt:   0%|          | 0.00/24.1G [00:00<?, ?B/s]

100%|██████████| 32/32 [00:16<00:00,  1.88it/s]

ATEN/CUDA backend not availabe. Make sure you install the hqq_aten library.
CPU times: user 1min 37s, sys: 1min 7s, total: 2min 45s
Wall time: 12min 16s


In [8]:
!nvidia-smi

Fri Aug 23 15:44:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   40C    P0              28W /  72W |  12997MiB / 23034MiB |      8%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [11]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [12]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [13]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [25]:

if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en



In [26]:
assessment = negative[0]
# assessment = positive[0]

In [27]:
%%time

if lang == "de":
  messages = [
    {"role": "user", "content": f'''
Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell.
Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

# Assessment
{assessment}

'''}
  ]
else:
  messages = [
    {"role": "user", "content": f'''
You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal.
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

CPU times: user 7min 2s, sys: 0 ns, total: 7min 2s
Wall time: 7min


Nein, es wird eine negative Empfehlung gegeben. Die aktuell genannte Diagnose als verordnungsbegründend erachtet, lässt keinen konkreten Befund erkennen. Zur fundierten Beurteilung der Erfordernis einer Krankenbehandlung oder -versorgung ist jedoch ein klar definierter medizinischer Befund unabdingbar.

In [28]:
def eval_assessment(assessment):
  if lang == "de":
    yes = "Ja"
    no = "Nein"
    messages = [
    {"role": "user", "content": f'''
Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell.
Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

# Assessment
{assessment}

'''}
  ]
  else:
    yes = "Yes"
    no = "No"
    messages = [
    {"role": "user", "content": f'''
You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal.
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''}
    ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith(yes):
    return "Positive", result
  elif result.startswith(no):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [29]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.
Negative: Nein, es wird eine negative Empfehlung gegeben. Die aktuell genannte Diagnose als verordnungsbegründend erachtet, lässt keinen konkreten Befund erkennen. Zur fundierten Beurteilung der Erfordernis einer Krankenbehandlung oder -versorgung ist jedoch ein klar definierter medizinischer Befund unabdingbar.
-----
Assessment: Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.
Neutral: Aufgrund der vorgelegten Leistungsauszüge der Krankenkasse wird eine negative Empfehlung für den beantragten Einsatz des funktionellen Produkts ausgesprochen, da der Versicherte gemäß den vorgelegten Informationen bereits entsprechend seinem Einsatzbereich versorgt ist. Die Versorgung mit dem beantragten funktionellen Produkt wäre demnach nicht notwendig und nicht gerechtfertigt.
-----
A

## Positive

In [30]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.
Positive: Ja, eine positive Empfehlung wird abgegeben.
Bei der genannten Diagnose ist der Einsatz eines Ausgleichs zur Sicherstellung des Grundbedürfnisses angemessen, da es sich wahrscheinlich um eine Erkrankung handelt, die eine erhöhte Unterstützung erfordert. Diese Maßnahme kann dazu beitragen, die Lebensqualität und die Gesundheit des Versicherten zu erhalten oder zu verbessern.
-----
Assessment: Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.
Positive: Ja, es wird eine positive Empfehlung ausgesprochen.

Die gesetzliche Krankenversicherung ist dazu verpflichtet, die Kosten für Hilfsmittel zu übernehmen, wenn diese medizinisch notwendig sind und eine Leistungspflicht der Krankenkasse besteht. Da die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung erfüllt sind, ist die positive 

In [31]:
!nvidia-smi

Fri Aug 23 18:17:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   75C    P0              70W /  72W |  13513MiB / 23034MiB |     96%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [32]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,Aus der aktuell als verordnungsbegründend bena...,Negative,Negative,"Nein, es wird eine negative Empfehlung gegeben..."
1,Gemäß den Leistungsauszügen der Krankenkasse i...,Negative,Neutral,Aufgrund der vorgelegten Leistungsauszüge der ...
2,"Eine medizinisch nachvollziehbare Begründung, ...",Negative,Negative,"Nein, es wird keine positive Empfehlung für di..."
3,In der Gesamtschau der hier vorliegenden Infor...,Negative,Negative,"Nein, es wird keine positive Empfehlung hinsic..."
4,"Eine ärztliche Begründung, warum im vorliegend...",Negative,Negative,"Nein, es wird keine positive Empfehlung gegebe..."
5,Bei der hier benannten Diagnose ist das Erford...,Positive,Positive,"Ja, eine positive Empfehlung wird abgegeben.\n..."
6,Die sozialmedizinischen Voraussetzungen für di...,Positive,Positive,"Ja, es wird eine positive Empfehlung ausgespro..."
7,Alltagsrelevante Gebrauchsvorteile werden fest...,Positive,Positive,"Ja, eine positive Empfehlung wird gegeben. Die..."
8,Sozialmedizinische Indikation für das Hilfsmit...,Positive,Positive,"Ja, es wird eine positive Empfehlung ausgespro..."
9,"Kontraindikationen wurden ausgeschlossen, es l...",Positive,Positive,"Ja, es wird eine positive Empfehlung für die V..."


In [33]:
df.to_excel(f'results_{kind}_{lang}.xlsx', index=False)

In [34]:
!ls -l

total 24
drwxr-xr-x 5 root root 4096 Aug 23 15:42 models--mobiuslabsgmbh--Mixtral-8x7B-Instruct-v0.1-hf-attn-4bit-moe-2bitgs8-metaoffload-HQQ
-rw-r--r-- 1 root root 6910 Aug 23 18:17 results_Mixtral-8x7B_de.xlsx
-rw-r--r-- 1 root root 7043 Aug 23 16:50 results_Mixtral-8x7B_en.xlsx
drwxr-xr-x 1 root root 4096 Aug 21 13:28 sample_data
